In [9]:
import os, json
import re
import pandas as pd
import numpy as np
import glob
import csv
import warnings
from pandas import isnull

# from google.colab import drive
# drive.mount('/content/drive')

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

pattern = r'(_d[1-8][ab])'
mapped_seq_df = pd.DataFrame()
list_of_sequences = []

# PLEASE CHANGE THE PATH ACCORDINGLY
your_path = '/Users/surya/Documents/UoM_Dissertation/'

# /Users/surya/Documents/UoM_Dissertation/artifacts/data/raw/Interaction Data

raw_path = your_path + 'artifacts/data/raw/'
preprocess_path =  your_path + 'artifacts/data/preprocessing/'
prediction_path = your_path + 'artifacts/data/prediction/'
json_path = raw_path + 'Interaction Data/'

print(json_path)
json_files = os.listdir(json_path)

unwanted_columns = [ 'screen', 'client', 'episodeCount',
       'sessionstartms', 'timezoneOffset', 'sd', 'sid', 'url', 'urlFull',
       'scroll', 'mouse', 'delta', 'key', 'change', 'select'
       'link', 'class', 'img', 'type', 'inheritedId', 'name', 'textValue', 'text', 'textContent']

necessary_columns = ['event', 'timestampms','node', 'user','id', 'dom', 'flag', 'task', 'merged_event']

total_columns = ['_id', 'event', 'screen', 'client', 'episodeCount', 'timestampms',
       'sessionstartms', 'timezoneOffset', 'sd', 'sid', 'url', 'urlFull',
       'mouse', 'node', 'scroll', 'delta', 'key', 'select', 'change', 'user',
       'type', 'textContent', 'text', 'link', 'inheritedId', 'img', 'class',
       'id', 'textValue', 'name', 'dom', 'flag', 'task', 'merged_event']

id_values = ['toDashboard1', 'toDashboard2', 'toDashboard3', 'toDashboard4', 'toDashboard5',
             'toDashboard6', 'toDashboard7', 'toDashboard8', 'toDashboard9', 'toDashboard10',
             'toDashboard11', 'toDashboard12', 'toDashboard13', 'toDashboard14', 'toDashboard15',
             'toDashboard16']

id_value_until_practice_tutorial = id_values[0]

surya_df = pd.DataFrame()

/Users/surya/Documents/UoM_Dissertation/artifacts/data/raw/Interaction Data/


### function definitions


In [10]:
### function definitions

def reading_json_files(json_data):
  json_data = re.sub(r'ObjectId\((.+?)\)', r'\1', json_data)
  tempdf = pd.read_json(json_data, lines=True)
  return tempdf


def basic_cleaning(tempdf):
  tempdf['user'] = i
  # tempdf.node = tempdf.node.map(lambda x: {} if isnull(x) else x)
  # tempdf.dropna(subset=['node'], inplace=True)
  # # tempdf.drop(tempdf.loc[:, 'screen':'urlFull'].columns, axis=1, inplace = True)
  tempdf.drop(['mobileTouch','evID'], axis=1, inplace = True, errors='ignore')
  return tempdf

def splitting_json_column_values(df):
      # Split the key-value pairs in the node column
      df['node'] = df['node'].apply(lambda x: {} if pd.isnull(x) else x)  # Convert NaN values to empty dictionaries
      # Extract the keys from the node dictionary
      keys = set()
      for node_dict in df['node']:
          keys.update(node_dict.keys())
      # Create new columns for each key
      for key in keys:
          df[key] = [node_dict.get(key) if isinstance(node_dict, dict) else None for node_dict in df['node']]

def updating_dom_element(df):
  count = df[df['dom'].notna() & df['dom'].str.contains('\)')].shape[0]
  # Strip the substring before ')' for non-NA values
  df.loc[df['dom'].notna() & df['dom'].str.contains('\)'), 'dom'] = df['dom'].str.split('\)').str[-1]
  df.reset_index(drop=True, inplace=True)

def removing_rows_of_practice_tutorial(df):
  # filling NA values
  df['id'].fillna(-555, inplace=True)
  # ignoring the indices until the practise tutorial
  last_index = df[df['id'] == id_value_until_practice_tutorial].index[-1]
  # Step 2: Remove all rows before the last index
  df = df.loc[last_index:]
  # Reset the index if desired
  df = df.reset_index(drop=True)

def removing_mid_values_between_toDashboardN_ids(df):
  # Remove mid rows between 'toDashboardN' first and last occurrences
  first_occurrences = {}
  last_occurrences = {}
  for value in range(1, 17):
      id_value = f'toDashboard{value}'
      first_occurrences[id_value] = df[df['id'] == id_value].index[0]
      last_occurrences[id_value] = df[df['id'] == id_value].index[-1]
  # Step 2: Remove the rows between the first and last occurrence for each value
  for value, first_index in first_occurrences.items():
      last_index = last_occurrences[value]
      df = df.drop(index=range(first_index , last_index))

def dropping_mid_rows_between_toDashboardN_ids(df):
  dropped_rows_count = {id_value: 0 for id_value in id_values}
  for id_value in id_values:
      # Get the indices of the first and last occurrence of the id value
      first_occurrence = df.index[df['id'] == id_value].min()
      last_occurrence = df.index[df['id'] == id_value].max()
      # Determine the indices of the middle rows between the first and last occurrence
      middle_rows_indices = list(range(first_occurrence , last_occurrence))
      dropped_rows_count[id_value] = len(middle_rows_indices)
      df.drop(middle_rows_indices, inplace=True)

  # Reset the index of the dataframe
  df.reset_index(drop=True, inplace=True)

def flagging_toDashboard_rows(df):
  df['flag'] = False
  # Step 2: Set the flag to True for rows matching the 'toDashboard' values
  for value in range(1, 17):
      id_value = f'toDashboard{value}'
      df.loc[df['id'] == id_value, 'flag'] = True
      df.reset_index(drop= True, inplace=True)

def merging_events(df):
    df['merged_event'] = df['event']
    # Replace 'mousewheel' with 'scroll'
    df['merged_event'] = df['merged_event'].replace('mousewheel', 'scroll')
    # Merge rows based on the initial conditions
    # df.loc[(df['merged_event'] == 'mouseover') & (df['merged_event'].shift(1) == 'mouseout'), 'merged_event'] = 'hover'
    df['merged_event'] = df['merged_event'].str.replace('mouseover', 'hover').str.replace('mouseout', 'hover')

    df['merged_event'] = df['merged_event'].str.replace('mouseup', 'click').str.replace('mousedown', 'click')
    # df.loc[(df['merged_event'] == 'mouseup') & (df['merged_event'].shift(1) == 'mousedown'), 'merged_event'] = 'click'
    df['merged_event'] = df['merged_event'].str.replace('keyup', 'press').str.replace('keydown', 'press')
    df['merged_event'] = df['merged_event'].str.replace('mobileTouchStart', 'click').str.replace('mobileTouchEnd', 'click')

    # df.loc[(df['merged_event'] == 'keyup') & (df['merged_event'].shift(1) == 'keydown'), 'merged_event'] = 'press'
    df.reset_index(drop=True, inplace=True)

def merging_multiple_events(df):
    df['multi_merged_event'] = df['merged_event']
    # Merge consecutive rows based on the additional conditions
    df.loc[(df['merged_event'].shift(1) == 'hover') & (df['merged_event'] == 'hover'), 'multi_merged_event'] = 'multihover'
    df.loc[(df['merged_event'].shift(1) == 'click') & (df['merged_event'] == 'click'), 'multi_merged_event'] = 'multiclick'
    df.loc[(df['merged_event'].shift(1) == 'press') & (df['merged_event'] == 'press'), 'multi_merged_event'] = 'multipress'
    df.loc[(df['merged_event'].shift(1) == 'scroll') & (df['merged_event'] == 'scroll'), 'multi_merged_event'] = 'multiscroll'
    df.reset_index(drop=True, inplace=True)

def merging_events_master(df):
    df['event_master'] = df['multi_merged_event']
    # Merge consecutive rows based on the additional conditions
    df.loc[(df['multi_merged_event'].shift(1) == 'mouseout') & (df['multi_merged_event'] == 'multihover'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'mouseout') & (df['multi_merged_event'] == 'hover'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'hover') & (df['multi_merged_event'] == 'mouseout'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'multihover') & (df['multi_merged_event'] == 'mouseout'), 'event_master'] = 'multihover'
    df.reset_index(drop=True, inplace=True)

def dropping_multi_hoverings(df):
  df['m_event'] = df['event_master'].ne(df['event_master'].shift()).cumsum()
  df = df[df['event_master'] != 'multihover']  # Remove 'mhover' rows
  df = df.drop_duplicates(subset='m_event', keep='first').drop(columns='m_event')


def initialising_new_column_task(df):
  df['task'] = df['id'].str.extract(pattern)
  df.loc[df['flag'], 'task'] = True
  df.reset_index(drop=True, inplace=True)

def getting_unique_task_values(df_name):
      temp_df = df_name  # Access the DataFrame dynamically using globals()
      # Step 1: Find unique non-NaN values in the 'task' column
      unique_values = temp_df['task'].unique()
      unique_task = next((x for x in unique_values if isinstance(x, str)), None)
      # Step 2: Check if any unique value is a string
      string_pattern = None
      return unique_task

def ignoring_na_dom_values(df):
  df = df[(df['dom'].notna()) | (df['flag'] == True)]
  df.reset_index(drop=True, inplace=True)

def getting_dataframes_task_wise_index(df):
  indices = df.index[df['flag']]
  for i in range(len(indices)-1):
      start = indices[i]
      end = indices[i+1]
      df_name = f"df_db{i+1}"
      globals()[df_name] = df.loc[start:end-1].copy()
  # Create the last DataFrame df_db16 from the last True value to the last row
  start = indices[-1]
  end = df.index[-1]
  df_db16 = df.loc[start:end-1].copy()
  task_wise_split_list = []
  # Step 3: Print the separate DataFrames
  for i in range(1, 16):
      df_name = f"df_db{i}"
      task_wise_split_list.append(globals()[df_name])
  task_wise_split_list.append(df_db16)
  return task_wise_split_list

def dropping_na_values_from_itemset_columns(df_name):
  df_name.loc[df_name['dom'] == '', 'dom'] = np.nan
  df_name.dropna(subset=['dom'], inplace=True)

def dropping_consecutive_rows(df, old_col, new_col):
  consecutive_duplicates = (df['final_event'] == df['final_event'].shift(1))
  # Drop duplicates, keeping only the first occurrence of each consecutive series
  df = df[~consecutive_duplicates]

def dropping_single_transformed_events(df):
  rows_to_remove = df[df['merged_event'] != df['event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)
  df = df.reset_index(drop=True)

def dropping_multi_transformed_events(df):
  rows_to_remove = df[df['multi_merged_event'] != df['merged_event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)

def dropping_events_master(df):
  rows_to_remove = df[df['event_master'] != df['multi_merged_event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)

def defining_sequence(series_list):
  # Concatenate all the series into a single series
  concatenated_series = pd.concat(series_list, ignore_index=True)
  # Append -2 to the end of the final series
  concatenated_series = concatenated_series.append(pd.Series([-2]))
  # removing rows with substring of nan values or index True values from 'Task' column
  sequence = concatenated_series[~concatenated_series.apply(lambda x: str(x).endswith(('Tru_e')))]
  return sequence

def replace_unique_strings_with_numbers(big_string):
    # Split the big string into words and remove duplicates
    words = big_string.split()
    unique_strings = list(set(words))
    # Remove '-1' and '-2' from the list of unique strings
    unique_strings = [s for s in unique_strings if s not in ['-1', '-2']]
    # Create a mapping dictionary between original string values and numbers
    num_mapping = {}
    next_number = 1  # Start assigning numbers from 1000
    for unique_string in unique_strings:
        num_mapping[unique_string] = str(next_number)
        next_number += 1
    # Replace the unique string values in the big string with their corresponding numbers
    for i in range(len(words)):
        if words[i] in num_mapping:
            words[i] = num_mapping[words[i]]
    # Join the words back into a new string
    new_string = ' '.join(words)
    # Create a DataFrame to store the mapping between the original and new string values
    mapping_df = pd.DataFrame(list(num_mapping.items()), columns=['Original_String', 'New_Number'])
    return new_string, mapping_df

def split_string_on_negative_two(input_string):
    # Split the input_string whenever "-2" is encountered
    output_string = re.sub(r'-2\s', '-2\n', input_string)
    return output_string

In [11]:
def defining_itemset(df_name, val):
  df_name['task_val'] =  val
  dropping_na_values_from_itemset_columns(df_name)
  dropping_single_transformed_events(df_name)
  merging_multiple_events(df_name)
  dropping_multi_transformed_events(df_name)
  merging_events_master(df_name)
  dropping_events_master(df_name)

  df_name['itemset'] = df_name['event_master'].astype(str) + "_" + df_name['type'].astype(str)
  # df_name['itemset'] = df_name['merged_event'].astype(str) + "_" + df_name['id'].astype(str) + df_name['dom'].astype(str)
  return df_name['itemset']

def deep_cleaning(d_t_list):
  itemset_column = []
  user_sequence = []
  abc_list = []
  for inner_df in d_t_list:
    it = defining_itemset(inner_df, getting_unique_task_values(inner_df))
    itemset_column.append(it.append(pd.Series([-1])))
    abc_list.append(inner_df)
  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  # Step 2: Convert the concatenated Series to a single line string
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"
  # user_sequence.append(defining_sequence(itemset_column))
  return single_line, abc_list


def major_cleaning(d_list):
  d_t_list = []
  one_concatened_dt_list = []
  master_line = ''
  for df in d_list:
      splitting_json_column_values(df)
      updating_dom_element(df)
      removing_rows_of_practice_tutorial(df)
      dropping_mid_rows_between_toDashboardN_ids(df)

      flagging_toDashboard_rows(df)
      initialising_new_column_task(df)
      getting_unique_task_values(df)
      merging_events(df)
      ignoring_na_dom_values(df)
      d_t_list = getting_dataframes_task_wise_index(df)
      # reforming the dataframe
      line, abc_list = deep_cleaning(d_t_list)
      abc_df = pd.concat(abc_list, ignore_index=True)
      one_concatened_dt_list.append(abc_df)
      master_line = master_line + line
  surya_df = pd.concat(one_concatened_dt_list, ignore_index=True)
  # list_of_sequences.append(d_t_list)
  return master_line, surya_df


In [12]:
def combining_similar_types(one):
  one['up_type'] = one['type']

  type_text = ['B', 'STRONG', 'I', 'P', 'GRAMMARLY-MIRROR', 'GRAMMARLY-EXTENSION', 'CENTER' , 'A']
  type_list = ['LI', 'UL']
  type_heading = ['H2', 'H3', 'H4', 'H5', 'HR']
  type_table = ['TD', 'TH']
  type_button = ['COM-1PASSWORD-BUTTON']
  type_graphic = ['rect', 'tspan', 'path', 'IMG']
  type_basic = ['HTML', 'BODY']
  type_select = ['SELECT', 'OPTION']


  # Use .loc to update 'up_type' column where 'type' column values match the values_to_update
  one.loc[one['type'].isin(type_text), 'up_type'] = 'TEXT'
  one.loc[one['type'].isin(type_heading), 'up_type'] = 'HEADING'
  one.loc[one['type'].isin(type_table), 'up_type'] = 'TABLE'
  one.loc[one['type'].isin(type_button), 'up_type'] = 'BUTTON'
  one.loc[one['type'].isin(type_graphic), 'up_type'] = 'GRAPHIC'
  one.loc[one['type'].isin(type_basic), 'up_type'] = 'BASIC'
  one.loc[one['type'].isin(type_select), 'up_type'] = 'SELECTION'
  one.loc[one['type'].isin(type_list), 'up_type'] = 'LISTING'

  return one

def find_consecutive_pairs(df):
    # Create a new column 'consecutive_pair' to store consecutive pairs
    df['consecutive_master_pair'] = df['event_master'] + ' -> ' + df['event_master'].shift(-1)
    df['consecutive_itemset_pair'] = df['itemset'] + ' -> ' + df['itemset'].shift(-1)
    # Drop the last row as it will have a NaN value for the 'shift(-1)' operation
    df.drop(df.index[-1], inplace=True)
    # Print the consecutive pairs of values in 'event' column
    merged_consecutive_pairs = df['consecutive_master_pair'].unique()

def remove_consecutive_duplicates(df, column_name, event):
    mask = (df[column_name] != event) | (df[column_name].shift() != event)
    new_df = df[mask].copy()
    while True:
        mask = (new_df[column_name] != event) | (new_df[column_name].shift() != event)
        temp_df = new_df[mask]
        if temp_df.equals(new_df):
            break
        new_df = temp_df.copy()
    return new_df

def replace_consecutive_events(df, column_name, old_event, new_event):
    mask = (df[column_name] == old_event) & (df[column_name].shift() == old_event)
    df.loc[mask, column_name] = new_event
    return df

def merge_click_sequences_recursive(df, column_name, event, new_event):
    mask = (df[column_name] != event) | (df[column_name].shift() != event)
    df = df[mask]
    if any(df[column_name] == event):
        df = replace_consecutive_events(df, column_name, event, new_event)
        return merge_click_sequences_recursive(df, column_name, event, new_event)
    else:
        return df

def common_values_across_db(series_list):
  # Initialize a set with the values from the first series
  common_values = set(series_list[0])
  # Iterate through the remaining series and update the common_values set
  for series in series_list[1:]:
      common_values.intersection_update(series)
  # Convert the common_values set back to a list if needed
  common_values_list = list(common_values)

  # Print the common values
  print("Common values:", common_values_list)

def user_mapping_with_timestamp(full):
  user_mapping = full[['user','sessionstartms']]
  list_time = user_mapping.sessionstartms.unique()
  data = { 'User': np.arange(1, len(list_time) + 1), 'Timestamp': list_time }
  user_time = pd.DataFrame(data)
  user_time.to_csv(preprocess_path+'usertimee.csv',index=False)


In [13]:
d_list = [None] * len(json_files)
# d_list = [None] * 10
i = 1

for file in json_files:
    with open(json_path + file, 'r') as fi:
        json_data = fi.read()
    tempdf = basic_cleaning(reading_json_files(json_data))
    d_list[i-1] = tempdf
    i = i+1
    # if i>10:
    #   break


In [ ]:
master_df = pd.concat(d_list, ignore_index=True)
master_df.info()

In [ ]:
master_df.url.value_counts()

In [ ]:
master_df.timezoneOffset.value_counts()

In [ ]:
master_df.describe()

In [ ]:
master_df.event.value_counts()

In [14]:
file_value, full = major_cleaning(d_list)
user_mapping_with_timestamp(full)
full.head()

,_id,event,screen,client,episodeCount,timestampms,sessionstartms,timezoneOffset,sd,sid,url,urlFull,mouse,node,scroll,delta,key,change,select,user,id,class,type,inheritedId,img,textValue,name,textContent,dom,flag,task,merged_event,task_val,multi_merged_event,event_master,itemset,link,text
0,6164104f4622cf0027242b69,mouseover,NaN,NaN,1,2021-10-11 10:22:07.300,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,"{'coordX': 2445, 'coordY': 1253, 'offsetX': 24...","{'dom': 'id(""tab-2855-6"")/DIV[1]/DIV[1]/DIV[1]...",NaN,NaN,NaN,NaN,NaN,1,tab-2855-6,[content-wrapper],DIV,True,None,None,None,\n,/DIV[1]/DIV[1]/DIV[1],False,NaN,hover,_d7b,hover,hover,hover_DIV,NaN,NaN
1,616410514622cf0027242b6a,change,NaN,NaN,1,2021-10-11 10:22:09.362,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,NaN,"{'dom': 'id(""dr_d7b"")/DIV[1]/INPUT[1]', 'id': ...",NaN,NaN,NaN,"{'type': 'text', 'value': '2020-01-27'}",NaN,1,dr_d7b,[form-control],INPUT,True,None,2020-01-27,None,null,/DIV[1]/INPUT[1],False,_d7b,change,_d7b,change,change,change_INPUT,NaN,NaN
2,616410994622cf0027242c82,mouseout,NaN,NaN,1,2021-10-11 10:23:21.387,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,"{'coordX': 1312, 'coordY': 925, 'offsetX': 126...","{'dom': 'id(""tab-2855-6"")/DIV[1]/DIV[1]/DIV[1]...",NaN,NaN,NaN,NaN,NaN,1,tab-2855-6,[row],DIV,True,None,None,None,\n,/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[5]/DIV[1]...,False,NaN,hover,_d7b,multihover,multihover,multihover_DIV,NaN,NaN
3,616410994622cf0027242c84,mousewheel,NaN,NaN,1,2021-10-11 10:23:21.483,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,NaN,"{'dom': 'id(""tab-2855-6"")/DIV[1]/DIV[1]/DIV[1]...",NaN,-18.691667,NaN,NaN,NaN,1,tab-2855-6,[row],DIV,True,None,None,None,\n,/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[5]/DIV[1]...,False,NaN,scroll,_d7b,scroll,scroll,scroll_DIV,NaN,NaN
4,616410b24622cf0027242d3c,mouseout,NaN,NaN,1,2021-10-11 10:23:46.017,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,"{'coordX': 2937, 'coordY': 164, 'offsetX': 65,...","{'dom': 'id(""id1_d7b"")/DIV[1]/LABEL[1]/SPAN[1]...",NaN,NaN,NaN,NaN,NaN,1,id1_d7b,[],SPAN,True,None,None,None,Random 2,/DIV[1]/LABEL[1]/SPAN[1],False,_d7b,hover,_d7b,multihover,multihover,multihover_SPAN,NaN,NaN


In [ ]:
full.info()

In [ ]:
full.type.value_counts()

In [16]:
final_string, mapping_dataframe = replace_unique_strings_with_numbers(file_value)
output_string = split_string_on_negative_two(final_string)
# Create and write the output to a text file named 'output.txt'
with open(preprocess_path +  'one.txt', 'w') as file:
    file.write(output_string)
# mapping_dataframe

CSV file

In [17]:
find_consecutive_pairs(full)


In [19]:
one = full[['event_master','type', 'itemset', 'id','class','dom','user','task_val', 'consecutive_master_pair', 'consecutive_itemset_pair']]
new_df = remove_consecutive_duplicates(combining_similar_types(one), 'event_master', 'multihover')
new_df.dropna(subset=['task_val'], inplace=True)
find_consecutive_pairs(new_df)


/var/folders/pd/5c5685_16pg8rhq7k0050lt80000gn/T/ipykernel_31710/1358475533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one['up_type'] = one['type']


In [20]:
new_df

,event_master,type,itemset,id,class,dom,user,task_val,consecutive_master_pair,consecutive_itemset_pair,up_type
0,hover,DIV,hover_DIV,tab-2855-6,[content-wrapper],/DIV[1]/DIV[1]/DIV[1],1,_d7b,hover -> change,hover_DIV -> change_INPUT,DIV
1,change,INPUT,change_INPUT,dr_d7b,[form-control],/DIV[1]/INPUT[1],1,_d7b,change -> multihover,change_INPUT -> multihover_DIV,INPUT
2,multihover,DIV,multihover_DIV,tab-2855-6,[row],/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[5]/DIV[1]...,1,_d7b,multihover -> scroll,multihover_DIV -> scroll_DIV,DIV
3,scroll,DIV,scroll_DIV,tab-2855-6,[row],/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[5]/DIV[1]...,1,_d7b,scroll -> multihover,scroll_DIV -> multihover_SPAN,DIV
4,multihover,SPAN,multihover_SPAN,id1_d7b,[],/DIV[1]/LABEL[1]/SPAN[1],1,_d7b,multihover -> click,multihover_SPAN -> click_INPUT,SPAN
...,...,...,...,...,...,...,...,...,...,...,...
17287,multihover,DIV,multihover_DIV,tab-9738-21,[row],/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[2],50,_d4b,multihover -> scroll,multihover_DIV -> scroll_SECTION,DIV
17288,scroll,SECTION,scroll_SECTION,tab-9738-21,[content],/DIV[1]/DIV[1]/DIV[1]/SECTION[1],50,_d4b,scroll -> multihover,scroll_SECTION -> multihover_DIV,SECTION
17289,multihover,DIV,multihover_DIV,line7_d4b,"[user-select-none, svg-container]",/DIV[1]/DIV[1],50,_d4b,multihover -> scroll,multihover_DIV -> scroll_DIV,DIV
17290,scroll,DIV,scroll_DIV,line7_d4b,"[user-select-none, svg-container]",/DIV[1]/DIV[1],50,_d4b,scroll -> multihover,scroll_DIV -> multihover_DIV,DIV


In [21]:
sorted_df = new_df.sort_values(by=['user','task_val'])
unique_task_val = sorted_df['task_val'].unique()
unique_user_val = sorted_df['user'].unique()

# Create an empty list to store the DataFrames
dataframe_sorted_db_list = []
dataframe_sorted_user_db_list = []

db_user_list = []
for user in unique_user_val:
    user_task_df = sorted_df[sorted_df['user'] == user].copy()
    dataframe_sorted_user_db_list.append(user_task_df)

mapped_dashboard_unique_itemsets = []
user_dashboard_wise_seq = []


In [22]:
full_seq = ''
for df in dataframe_sorted_user_db_list:
  itemset_column = []
  concatenated_series = pd.Series()
  unique_its = []
  for task in unique_task_val:
    task_df = df[df['task_val'] == task].copy()
    task_df['its'] = task_df['event_master'].astype(str) + '__' + task_df['up_type'].astype(str)
    unique_its.append(task_df['its'])
    itemset_column.append(task_df['its'].append(pd.Series([-1])))
  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"
  full_seq = full_seq + single_line


In [23]:
final, mapping_ut_dataframe = replace_unique_strings_with_numbers(full_seq)
output_full_string = split_string_on_negative_two(final)
# Create and write the output to a text file named 'output.txt'
with open(preprocess_path+'full_m_2.txt', 'w') as file:
    file.write(output_full_string)
mapping_ut_dataframe

,Original_String,New_Number
0,hover__HEADING,1
1,multiclick__HEADING,2
2,select__SELECTION,3
3,scroll__LISTING,4
4,contextmenu__SPAN,5
...,...,...
108,select__TABLE,109
109,scroll__SPAN,110
110,dblclick__TABLE,111
111,scroll__TABLE,112


In [24]:
mapping_ut_dataframe.to_csv(preprocess_path+"Mapped_values.csv")

In [25]:
# surya

sorted_df = new_df.sort_values(by=['task_val','user'])
unique_task_val = sorted_df['task_val'].unique()
unique_user_val = sorted_df['user'].unique()

dataframe_sorted_db_list = []
dashboard_wise_seq = []

# db_user_list = []
for task in unique_task_val:
    user_task_df = sorted_df[sorted_df['task_val'] == task].copy()

    dataframe_sorted_db_list.append(user_task_df)

for df in dataframe_sorted_db_list:
  itemset_column = []
  concatenated_series = pd.Series()
  # unique_its = []
  for user_no in unique_user_val:
    user_df = df[df['user'] == user_no].copy()
    user_df['its'] = user_df['event_master'].astype(str) + '__' + user_df['up_type'].astype(str) + '__' + user_df['user'].astype(str)
    itemset_column.append(user_df['its'].append(pd.Series([-1])))
  # itemset_column.append(unique_its.append())
  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"

  s1, m1 = replace_unique_strings_with_numbers(single_line)
  mapped_dashboard_unique_itemsets.append(m1)
  dashboard_wise_seq.append(s1)
  # common_values_across_db(unique_its)


In [ ]:
dataframe_sorted_db_list[0]

In [ ]:
mapped_dashboard_unique_itemsets[0]

In [ ]:
dashboard_wise_seq[0]

In [26]:
merged_string = ''.join(dashboard_wise_seq)
op_merged = split_string_on_negative_two(merged_string)
with open(preprocess_path+'op_merged.txt', 'w') as fi:
      fi.write(op_merged)

# dashboard_wise_seq[0]

# dataframe_sorted_db_list

In [27]:
# Function to write strings into new files
def write_strings_to_files(string_list):
    for i, string in enumerate(string_list):
        print(len(string))
        file_name = f"ab_{i+1}.txt"  # Assuming filenames like file_0.txt, file_1.txt, etc.
        with open(file_name, 'w') as file:
            file.write(string)

write_strings_to_files(dashboard_wise_seq)

3053
3794
4882
4545
4019
4631
4748
4615
2927
2972
3264
2859
5056
5271
5275
5148


In [28]:
mapped_dashboard_unique_itemsets

[             Original_String New_Number
 0      multiclick__BASIC__38          1
 1             scroll__DIV__9          2
 2    multihover__SECTION__36          3
 3             click__DIV__45          4
 4          dblclick__TEXT__1          5
 ..                       ...        ...
 446           hover__DIV__21        447
 447           hover__DIV__27        448
 448           click__DIV__36        449
 449  multihover__GRAPHIC__48        450
 450          click__SPAN__13        451
 
 [451 rows x 2 columns],
              Original_String New_Number
 0         click__SECTION__46          1
 1            hover__SPAN__50          2
 2      multiclick__BASIC__38          3
 3    multihover__SECTION__36          4
 4        scroll__HEADING__39          5
 ..                       ...        ...
 513        select__TABLE__39        514
 514           hover__DIV__21        515
 515     multihover__TEXT__25        516
 516     multiscroll__DIV__40        517
 517          click__SPAN__13 

In [30]:
new_df.to_csv(preprocess_path+"two.csv", index=False)


In [31]:
find_consecutive_pairs(new_df)

In [32]:
new_df.consecutive_master_pair.value_counts()

multihover -> click          2435
click -> multihover          2153
scroll -> multihover         1474
multihover -> scroll         1412
multihover -> multiclick      716
                             ... 
press -> multiscroll            2
multiclick -> contextmenu       1
multihover -> contextmenu       1
multihover -> dblclick          1
dblclick -> multipress          1
Name: consecutive_master_pair, Length: 78, dtype: int64

In [33]:
new_df.consecutive_itemset_pair.value_counts()

multihover_SPAN -> click_SPAN           708
multihover_DIV -> scroll_DIV            427
click_SPAN -> multihover_SPAN           401
click_SPAN -> multihover_DIV            384
change_INPUT -> multihover_DIV          312
                                       ... 
press_BODY -> hover_SPAN                  1
hover_SPAN -> press_BODY                  1
select_SECTION -> multiclick_SECTION      1
multiclick_SECTION -> select_SECTION      1
scroll_STRONG -> multihover_SPAN          1
Name: consecutive_itemset_pair, Length: 1756, dtype: int64

In [34]:
full.event_master.value_counts()

multihover     6251
click          2995
hover          2172
scroll         2166
multiclick     1085
multiscroll     962
change          853
select          303
press           257
multipress      187
dblclick         58
contextmenu       4
Name: event_master, dtype: int64

In [ ]:
# masterq_df.info()

In [ ]:
full.info()

In [ ]:
user_mapping = full[['user','sessionstartms']]
list_time = user_mapping.sessionstartms.unique()
data = { 'User': np.arange(1, len(list_time) + 1), 'Timestamp': list_time }
user_time = pd.DataFrame(data)
user_time.to_csv(preprocess_path+'usertime.csv',index=False)

In [ ]:
cc

def get_unique_values_except_minus_one(input_lines):
    unique_values = set()

    for line in input_lines:
        values = line.split()
        for value in values:
            try:
                int_value = int(value)
                if int_value != -1:
                    unique_values.add(int_value)
            except ValueError:
                pass

    return unique_values

input_text = """
277 -1 692 -1 811 -1
277 -1 692 -1 417 -1
289 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 550 -1
289 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 550 -1
179 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 550 -1
289 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 550 -1
179 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 550 -1
289 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 550 -1
179 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 550 -1
289 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 222 -1
179 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 550 -1
289 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 222 -1
179 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 222 -1
289 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 222 -1
179 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 222 -1
289 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 222 -1
179 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 222 -1
179 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 222 -1
"""

input_lines = input_text.strip().split('\n')
unique_values = get_unique_values_except_minus_one(input_lines)

print("Unique values except -1:", unique_values)


In [ ]:
from collections import Counter
import re

input_text = """
277 -1 692 -1 811 -1
277 -1 692 -1 417 -1
289 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 550 -1
289 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 550 -1
179 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 550 -1
289 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 550 -1
179 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 550 -1
289 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 550 -1
179 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 550 -1
289 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 222 -1
179 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 550 -1
289 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 222 -1
179 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 222 -1
289 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 222 -1
179 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 222 -1
289 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 222 -1
179 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 222 -1
179 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 222 -1
"""


# Extract numbers from the input_text using regular expression
numbers = re.findall(r'\d+', input_text)

# Convert the numbers to integers and count their occurrences
number_counts = Counter(map(int, numbers))

data_count = {'New_Number': [], 'Count': []}
# Print the count of each unique value
for value, count in number_counts.items():
    # print(f"Value: {value}, Count: {count}")

    data_count['New_Number'].append(value)
    data_count['Count'].append(count)

df_data_count = pd.DataFrame(data_count)
df_data_count



In [ ]:
mapping_ut_dataframe['New_Number'] = mapping_ut_dataframe['New_Number'].astype('int64')
# mapping_ut_dataframe.info()

In [ ]:
df_data_count.info()

In [ ]:
merged_df_final = pd.merge(mapping_ut_dataframe, df_data_count, on='New_Number')
merged_df_final['Last_Three_Characters'] = merged_df_final['Original_String'].str[-3:]
merged_df_final.sort_values(by='Last_Three_Characters',inplace=True)
merged_df_final

In [ ]:
# sum_count = merged_df_final['Count'].sum()
# print("Sum of Count column:", sum_count)

In [ ]:
merged_df_final['Last_Three_Characters']

In [ ]:
cumulative_sum_df = merged_df_final.groupby('Last_Three_Characters')['Count'].cumsum()
merged_df_final['Cumulative_Count'] = cumulative_sum_df

merged_df_final


In [ ]:
highest_cumulative_counts = merged_df_final.groupby('Last_Three_Characters')['Cumulative_Count'].max()
print(highest_cumulative_counts.sort_values(ascending=False))

In [ ]:
15 222 336 391 934 -1

In [ ]:
from tabulate import tabulate

# Provided information
info_str = """
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1114761 entries, 0 to 1114760
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype
---  ------          --------------    -----
 0   _id             1090484 non-null  object
 1   event           1114761 non-null  object
 2   screen          50 non-null       object
 3   client          50 non-null       object
 4   episodeCount    1114761 non-null  int64
 5   timestampms     1114761 non-null  datetime64[ns]
 6   sessionstartms  1114761 non-null  int64
 7   timezoneOffset  1114761 non-null  int64
 8   sd              1114761 non-null  int64
 9   sid             464350 non-null   object
 10  url             1114761 non-null  object
 11  urlFull         1114761 non-null  object
 12  mouse           833011 non-null   object
 13  node            1082820 non-null  object
 14  scroll          30847 non-null    object
 15  delta           14266 non-null    float64
 16  key             197291 non-null   object
 17  select          2027 non-null     object
 18  change          34512 non-null    object
dtypes: datetime64[ns](1), float64(1), int64(4), object(13)
memory usage: 170.1+ MB
"""

# Process the info_str to extract relevant information
lines = info_str.strip().split('\n')
headers = lines[5].split()
data = [line.split() for line in lines[7:-1]]

# Convert the data into a dictionary for tabulate
info_dict = {
    "Column": [row[0] for row in data],
    "Non-Null Count": [row[1] for row in data],
    "Dtype": [row[3] for row in data]
}

# Convert the dictionary into a table using tabulate
table = tabulate(info_dict, headers="keys", tablefmt="grid")

print(table)


In [ ]:
from tabulate import tabulate
from docx import Document
from docx.shared import Inches

# Provided information
info_str = """
# ... (same as before)
"""

# Process the info_str to extract relevant information
lines = info_str.strip().split('\n')
headers = lines[5].split()
data = [line.split() for line in lines[7:-1]]

# Convert the data into a dictionary for tabulate
info_dict = {
    "Column": [row[0] for row in data],
    "Non-Null Count": [row[1] for row in data],
    "Dtype": [row[3] for row in data]
}

# Convert the dictionary into a table using tabulate
table = tabulate(info_dict, headers="keys", tablefmt="grid")

# Create a Word document
doc = Document()
doc.add_heading('DataFrame Information', level=1)

# Add the table to the document
table_paragraph = doc.add_paragraph(table)
table_paragraph.alignment = Inches(1)  # Set alignment if needed

# Save the document
doc.save('dataframe_info.docx')
